In [1]:
from petsc4py import PETSc
import numpy as np

# -----------------------------------------------------------------------------
# 1) Define the residual function F(u) = 0
# -----------------------------------------------------------------------------
def form_function(snes, X, F):
    """
    Compute F = F(u) at the current iterate u = X.
    Here u has length 2, so X.getArray() is [u1, u2].
    We want:
      F1 = sin(u1) - u2
      F2 = u1^2 + u2^2 - 1
    """
    u = X.getArray()
    u1, u2 = u[0], u[1]
    f1 = np.sin(u1) - u2
    f2 = u1**2 + u2**2 - 1.0

    # Set into PETSc vector F
    F.setArray(np.array([f1, f2], dtype=np.float64))


# -----------------------------------------------------------------------------
# 2) Define (optional) Jacobian application; here we can provide a matrix,
#    but we’ll just let SNES do finite differences automatically.  For
#    demonstration, we show how you might supply a Jacobian explicitly.
# -----------------------------------------------------------------------------
def form_jacobian(snes, X, J, P):
    """
    Compute J = dF/du at u = X.
    J[i,j] = ∂F_i/∂u_j.  Here:
      ∂F1/∂u1 =  cos(u1);    ∂F1/∂u2 = -1
      ∂F2/∂u1 = 2*u1;        ∂F2/∂u2 = 2*u2
    We assume X is length 2.  The PETSc Mat J is 2×2.
    """
    u = X.getArray()
    u1, u2 = u[0], u[1]

    # Build the 2×2 Jacobian
    J_mat = np.array([[ np.cos(u1),   -1.0 ],
                      [ 2*u1,          2*u2 ]], dtype=np.float64)

    # Insert into the PETSc Mat
    rows = [0, 1]
    cols = [0, 1]
    J.zeroEntries()
    for i in rows:
        for j in cols:
            J.setValue(i, j, J_mat[i, j])
    J.assemble()
    P.assemble()  # Usually P == J for SNES

    return


# -----------------------------------------------------------------------------
# 3) Set up SNES + KSP context
# -----------------------------------------------------------------------------
opts = PETSc.Options()
snes = PETSc.SNES().create(comm=PETSc.COMM_WORLD)

# Point SNES at our functions
snes.setFunction(form_function, Vec=PETSc.Vec().createSeq(2))
snes.setJacobian(form_jacobian,
                 J=PETSc.Mat().createDense([2, 2]),
                 P=PETSc.Mat().createDense([2, 2]))

# -----------------------------------------------------------------------------
# 4) Tell SNES to use NGMRES (nonlinear GMRES) instead of default Newton
#
#    In PETSc, this is "SNESNGMRES".  We can set it by name or via options.
#    The simplest is:
#         snes.setType('ngmres')
#
#    Or equivalently (via command‐line style):
#         opts['snes_type'] = 'ngmres'
# -----------------------------------------------------------------------------
snes.setType('ngmres')
#
# You can also tweak NGMRES‐specific options; for instance:
#    - How many previous iterates to keep:  -snes_ngmres_m <m>
#    - Which restart strategy, tolerances, etc.
#
# Example (if you want to pass these programmatically):
# opts['snes_ngmres_m'] = 5
# opts['snes_ngmres_select_type'] = 'linesearch'  # or 'difference', 'none', etc.
# -----------------------------------------------------------------------------

# Let SNES create its default KSP (which is GMRES for the linear subproblems)
snes.getKSP().setType('gmres')
snes.getKSP().getPC().setType('lu')  # or 'jacobi', 'ilu', etc.

# -----------------------------------------------------------------------------
# 5) Choose an initial guess and solve
# -----------------------------------------------------------------------------
u0 = PETSc.Vec().createSeq(2)
u0.setArray(np.array([0.5, 0.5], dtype=np.float64))  # some initial guess

snes.solve(None, u0)
if snes.getConvergedReason() < 0:
    print("SNES failed to converge, reason = ", snes.getConvergedReason())
else:
    sol = u0.getArray(readonly=True)
    print("NGMRES converged in %d SNES iterations; solution = %r" %
          (snes.getIterationNumber(), sol))


ModuleNotFoundError: No module named 'petsc4py'